In [1]:
from platform import python_version

print(python_version())

3.5.4


In [2]:
!pip install numpy==1.16.4
import numpy as np

     |████████████████████████████████| 17.2MB 3.1MB/s eta 0:00:01    |███████▌                        | 4.1MB 2.2MB/s eta 0:00:06            | 7.6MB 5.7MB/s eta 0:00:02
  Found existing installation: numpy 1.17.0
    Uninstalling numpy-1.17.0:
      Successfully uninstalled numpy-1.17.0
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pip

try:
    __import__('keras')
except ImportError:
    pip.main(['install', 'keras']) 
    
try:
    __import__('h5py')
except ImportError:
    pip.main(['install', 'h5py']) 

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

seed = 1337
np.random.seed(seed)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from keras.datasets import reuters

max_words = 1000

np_load_old = np.load
np.load = lambda *a: np_load_old(*a, allow_pickle=True)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2, seed=seed)
np.load = np_load_old


num_classes = np.max(y_train) + 1  # 46 topics




In [5]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [6]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [7]:
model = Sequential()  # Instantiate sequential model
model.add(Dense(512, activation ='relu', input_shape=(max_words,)))# Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) # Add second layer
model.add(Dense(num_classes, activation='softmax')) # Add third layer

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 32
model.fit(x_train, y_train, epochs=5, batch_size=batch_size)
score = model.evaluate(x_test,y_test,batch_size=batch_size)

Epoch 1/5
8982/8982 [==============================] - 24s - loss: 1.3934 - acc: 0.6900    
Epoch 2/5
8982/8982 [==============================] - 23s - loss: 0.7729 - acc: 0.8185    
Epoch 3/5
8982/8982 [==============================] - 19s - loss: 0.5553 - acc: 0.8654    
Epoch 4/5
8982/8982 [==============================] - 17s - loss: 0.4217 - acc: 0.8938    
Epoch 5/5
2208/2246 [============================>.] - ETA: 0s

In [9]:
score[1]

0.8023152270703473

In [10]:
model.save("model.h5")  # upload this file to the grader in the next code block

In [11]:
!base64 model.h5 > model.h5.base64

In [12]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2019-09-02 22:27:52--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving webproxy (webproxy)... 10.36.66.1
Connecting to webproxy (webproxy)|10.36.66.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2019-09-02 22:27:52 (6.16 MB/s) - ‘rklib.py’ saved [2540/2540]



In [13]:
# %load rklib.py
import requests, json
import zipfile
import os

def submit(submitterEmail,secret,key,submission_part, all_parts, data):
        submission = {
                    "assignmentKey": key,
                    "submitterEmail":  submitterEmail,
                    "secret":  secret,
                    "parts": {}
                  }
        for part in all_parts:
            if part == submission_part:
                submission["parts"][part] = {"output": data}
            else:
                submission["parts"][part] = dict()
        response = requests.post('https://www.coursera.org/api/onDemandProgrammingScriptSubmissions.v1', data=json.dumps(submission))
        if response.status_code == 201:
            print ("Submission successful, please check on the coursera grader page for the status")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")
        else:
            print ("Something went wrong, please have a look at the reponse of the grader")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")
            
def submitAll(submitterEmail,secret,key,parts_and_data):
        submission = {
                    "assignmentKey": key,
                    "submitterEmail":  submitterEmail,
                    "secret":  secret,
                    "parts": {}
                  }
        for part, output in parts_and_data.items():
            if output is not None:
                submission["parts"][part] = {"output": output}
            else:
                submission["parts"][part] = dict()
        response = requests.post('https://www.coursera.org/api/onDemandProgrammingScriptSubmissions.v1', data=json.dumps(submission))
        if response.status_code == 201:
            print ("Submission successful, please check on the coursera grader page for the status")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")
        else:
            print ("Something went wrong, please have a look at the reponse of the grader")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")


def zipit(target, path):
    zipf = zipfile.ZipFile(target, 'w', zipfile.ZIP_DEFLATED)
    for root, dirs, files in os.walk(path):
        for file in files:
            zipf.write(os.path.join(root, file))
    zipf.close()


In [14]:
key = "XbAMqtjdEeepUgo7OOVwng"
part = "LqPRQ"
email = "dennislamcv@gmail.com"
secret = "AEoxC9hTjjQpG9IH"

with open('model.h5.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"ozVf2","id":"tE4j0qhMEeecqgpT6QjMdA~ozVf2~8eTct83QEemH3Aqun-E-ng","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
